# Simulation performance assessment

This example runs several simulations at with different parameters, in order to measure computational performace measures of sparrowpy.
The performance metrics are written in local files and can be visualized in [``VIB_street_canyon_plots.ipynb``](VIB_street_canyon_plots.ipynb).

**NOTE:** These simulations are extremely time consuming and require a large amount of computational resources.

![alt text](figures/urban_canyon.svg "Illustration of the street canyon")

In [ ]:
"""Test the radiosity.Radiosity module."""
import pyfar as pf
import sparrowpy as sp
from utils.run_simu import run_simu, run_simu_mem
import json
import os

outpath = os.path.join(os.getcwd(),"resources","user")

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [33]:
def urban_canyon_stub(length_x, length_y, length_z):
    """Create a shoebox room with the given dimensions.

    Parameters
    ----------
    length_x : float
        Length of the urban canyon in meters.
    length_y : float
        width of the urban canyon in meters.
    length_z : float
        height of the urban canyon in meters.

    Returns
    -------
    room : list[geo.Polygon]
        List of the walls of the room.

    """
    return [
        sp.geometry.Polygon(
            [[0, 0, 0], [length_x, 0, 0],
            [length_x, 0, length_z], [0, 0, length_z]],
            [0, 0, 1], [0, 1, 0]),
        sp.geometry.Polygon(
            [[0, 0, 0],[0, length_y, 0],
            [length_x, length_y, 0], [length_x, 0, 0 ]],
            [1, 0, 0], [0, 0, 1]),
        sp.geometry.Polygon(
            [[0, length_y, 0], [length_x, length_y, 0],
            [length_x, length_y, length_z], [0, length_y, length_z]],
            [0, 0, -1], [0, -1, 0]),
        ]

Define basic scene properties

In [34]:
# Define parameters
X = 90
Y = 12
Z = 6

# create geometry
canyon = urban_canyon_stub(X, Y, Z)
source = pf.Coordinates(20, 6, 1)
receiver = pf.Coordinates(21, 6, 2)

Run a very basic scene: this caches optimized functions, ensuring best efficiency for the actual simulations.

In [ ]:
_=run_simu(walls=urban_canyon_stub(1,1,1),
        source=pf.Coordinates(.5,.5,.5),
        receiver=pf.Coordinates(.5,.5,.5),
        patch_size=1)

c:\Users\jotag\Documents\dev\sparapy\supp_materials\utils\run_simu.py:84: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in favor of spharpy.samplings.gaussian.'
  samples = pf.samplings.sph_gaussian(res)
c:\Users\jotag\Documents\dev\sparapy\.venv\Lib\site-packages\pyfar\samplings\samplings.py:344: PyfarDeprecationWarning: 'This function will be changed in pyfar 0.8.0 to init(x, y, z).'
  sampling = pyfar.Coordinates(
c:\Users\jotag\Documents\dev\sparapy\.venv\Lib\site-packages\pyfar\classes\coordinates.py:276: PyfarDeprecationWarning: 'This function will be deprecated in pyfar 0.8.0 in favor of spharpy.samplings.SamplingSphere.'
  self.sh_order = sh_order
c:\Users\jotag\Documents\dev\sparapy\supp_materials\utils\run_simu.py:101: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  radi.set_wall_brdf(


(TimeData:
 (1, 1) channels with 5 samples,
 [0.004546403884887695, 0.0024492740631103516, 0.0009505748748779297, 0.0])

Range of testing for different simulation parameters

In [ ]:
time_resolutions = [1/50,1/100,1/250,1/500,1/1000]
patch_sizes = [6,3,2,1.5]
refl_orders = [1,10,20,40,60,80,100]
BRDF_samples = [2,4,6,8,10]

Set baseline conditions and initialize output dataframe

In [ ]:
# baseline simulation parameters
patch_size_baseline = 3
max_order_baseline = 10
time_resolution_baseline = 1/100
BRDF_samples_baseline = 4

out={"baseline":{"patch_size":[],
                 "reflection_order":[],
                 "time_step":[],
                 "brdf_resolution":[]},
     "reflection_order":{"mem":[],
                  "t":[],
                  "edc":[],
                  "data":[]},
     "brdf_resolution":{"mem":[],
                  "t":[],
                  "edc":[],
                  "data":[]},
     "patch_size":{"mem":[],
                  "t":[],
                  "edc":[],
                  "data":[]},
     "time_step":{"mem":[],
                  "t":[],
                  "edc":[],
                  "data":[]},
                  }

out["baseline"]["patch_size"]=patch_size_baseline

out["baseline"]["reflection_order"]=max_order_baseline

out["baseline"]["time_step"]=time_resolution_baseline

out["baseline"]["brdf_resolution"]=BRDF_samples_baseline

Measure runtime effects

In [ ]:
# ETC time resolution

for ts in time_resolutions:
    edc,t = run_simu( walls=canyon, source=source,
                    receiver=receiver, patch_size=patch_size_baseline,
                    time_step=ts, duration=1.2,
                    refl_order=max_order_baseline, res=BRDF_samples_baseline)
    out["time_step"]["edc"]=edc.time.tolist()
    out["time_step"]["t"].append(t)
out["time_step"]["data"]=time_resolutions

# write to file
with open(os.path.join(outpath,'street_canyon_perf_no_mem.json'), 'w') as fp:
    json.dump(out, fp,indent=2)


In [ ]:
# max reflection order

for refl in refl_orders:
    edc,t = run_simu( walls=canyon, source=source,
                    receiver=receiver, patch_size=patch_size_baseline,
                    time_step=time_resolution_baseline, duration=1.2,
                    refl_order=refl, res=BRDF_samples_baseline)
    out["reflection_order"]["edc"]=edc.time.tolist()
    out["reflection_order"]["t"].append(t)
out["reflection_order"]["data"]=refl_orders

# write to file
with open(os.path.join(outpath,'street_canyon_perf_no_mem.json'), 'w') as fp:
    json.dump(out, fp,indent=2)



In [ ]:
# BRDF resolution
for ang in BRDF_samples:
    edc,t = run_simu( walls=canyon, source=source,
                    receiver=receiver, patch_size=patch_size_baseline,
                    time_step=time_resolution_baseline, duration=1.2,
                    refl_order=max_order_baseline, res=ang)
    out["brdf_resolution"]["edc"]=edc.time.tolist()
    out["brdf_resolution"]["t"].append(t)
out["brdf_resolution"]["data"]=BRDF_samples

# write to file
with open(os.path.join(outpath,'street_canyon_perf_no_mem.json'), 'w') as fp:
    json.dump(out, fp,indent=2)

In [ ]:
# patch sizes

for pat in patch_sizes:
    edc,t = run_simu( walls=canyon, source=source,
                    receiver=receiver, patch_size=pat,
                    time_step=time_resolution_baseline, duration=1.2,
                    refl_order=max_order_baseline, res=BRDF_samples_baseline )
    out["patch_size"]["edc"] = edc.time.tolist()
    out["patch_size"]["t"].append(t)
out["patch_size"]["data"]=patch_sizes

# write to file
with open(os.path.join(outpath,'street_canyon_perf_no_mem.json'), 'w') as fp:
    json.dump(out, fp,indent=2)

Measure memory consumption 

**EXTREMELY TIME CONSUMING**

In [ ]:
# memory
for ts in time_resolutions:
    out["time_step"]["mem"].append(run_simu_mem( walls=canyon,
                                                source=source,
                    receiver=receiver, patch_size=patch_size_baseline,
                    time_step=ts, duration=1.2,
                    refl_order=max_order_baseline, res=BRDF_samples_baseline))
print("time resolution done")

# write to file
with open(os.path.join(outpath,'street_canyon_perf.json'), 'w') as fp:
    json.dump(out, fp,indent=2)

for refl in refl_orders:
    out["reflection_order"]["mem"].append(run_simu_mem( walls=canyon,
                                                       source=source,
                    receiver=receiver, patch_size=patch_size_baseline,
                    time_step=time_resolution_baseline, duration=1.2,
                    refl_order=refl, res=BRDF_samples_baseline))
print("refl. order done")

# write to file
with open(os.path.join(outpath,'street_canyon_perf.json'), 'w') as fp:
    json.dump(out, fp,indent=2)

for ang in BRDF_samples:
    out["brdf_resolution"]["mem"].append(run_simu_mem( walls=canyon,
                                                      source=source,
                    receiver=receiver, patch_size=patch_size_baseline,
                    time_step=time_resolution_baseline, duration=1.2,
                    refl_order=max_order_baseline, res=ang))
print("BRDF samples done")

# write to file
with open(os.path.join(outpath,'street_canyon_perf.json'), 'w') as fp:
    json.dump(out, fp,indent=2)


In [ ]:
for pat in patch_sizes:
    out["patch_size"]["mem"].append(run_simu_mem( walls=canyon, source=source,
                    receiver=receiver, patch_size=pat,
                    time_step=time_resolution_baseline, duration=1.2,
                    refl_order=max_order_baseline, res=BRDF_samples_baseline))

# write to file
with open(os.path.join(outpath,'street_canyon_perf.json'), 'w') as fp:
    json.dump(out, fp,indent=2)

In [ ]:
%load_ext watermark
%watermark -v -m -iv